In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

# Any results you write to the current directory are saved as output.

In [2]:
os.listdir('../input/himu-book')

['himu.txt']

In [3]:
with open('../input/himu-book/himu.txt', 'r') as f:
    text = f.read()

In [4]:
text[:500]

'\nপ্রসঙ্গ হিমু\n\n\n\n\tহিমু আমার প্রিয় চরিত্রের একটি। যখন হিমুকে নিয়ে কিছু লিখি-- নিজেকে হিমু মনে হয়, একধরনের ঘোর অনুভব করি। এই ব্যাপারটা অন্য কোন লেখার সময় তেমন ঘটে না। হিমুকে নিয়ে আমার প্রথম লেখা ময়ুরাক্ষি। ময়ুরাক্ষি লেখার সময় ব্যাপারটা প্রথম লক্ষ করি। দ্বিতীয়বারে লিখলাম রজার ওপাশে। তখনো একই ব্যাপার। কেন এরকম হয়? মানুষ হিসেবে আমি যুক্তিবাদী। হিমু যুক্তিহীন, রহস্যময় জগৎ একজন যুক্তিবাদীকে কেন আকর্ষণ করবে? আমার জানা নেই। যদি কখনও জানতে পারি-- পাঠকদের জানাব।\n\n\n\n\tহুমায়ূন আহমেদ\n\n\tএলিফেন্ট রোড\n\n\n\n\n\nকি নাম'

In [5]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

encoded[:100]

array([  0,  41,  95,  36,  53,  29,  95,  81,  25,   1,  59,  99, 100,
         0,   0,   0,   0, 108,   1,  59,  99, 100,  25,   9,  99,  42,
        36,  25,  41,  95,  36,  59,  27,  25,  18,  36,  59,  16,  95,
        36,   2,  36,  25,  50,  93,  14,  59,  55,  25,  40,  24,   7,
        25,   1,  59,  99, 100,  93,   2,  25,   7,  59,  27,   2,  25,
        93,  59, 101, 100,  25,  63,  59,  24,  59,  83,  83,  25,   7,
        59,  17,   2,  93,   2,  25,   1,  59,  99, 100,  25,  99,   7,
         2,  25,   1,  27,  54,  25,  50,  93,   6])

In [6]:
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [8]:
def get_batches(arr, batch_size, seq_length):
    
    batch_size_total = batch_size * seq_length
    
    n_batches = len(arr)//batch_size_total
    arr = arr[:n_batches * batch_size_total]
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [9]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [10]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[  0  41  95  36  53  29  95  81  25   1]
 [ 42   7   2  25  90  42  16  25  24   2]
 [ 54  25  86  42  75  36 100  99   2  25]
 [ 27   2  25  50  53   2 101  59  63  42]
 [ 25   9 101   2  55  25   9  36  25  24]
 [100  40   2  42  81  25   9  41   7  42]
 [ 99  55  25  50  99  42  25   4  36  17]
 [  2  25   1  86   2  25  41  42 107  18]
 [ 25  93  36   2 101   2   7  55  25  16]
 [ 44   7  42  36  25  21  26 100   6  41]]

y
 [[ 41  95  36  53  29  95  81  25   1  59]
 [  7   2  25  90  42  16  25  24   2  16]
 [ 25  86  42  75  36 100  99   2  25  70]
 [  2  25  50  53   2 101  59  63  42  99]
 [  9 101   2  55  25   9  36  25  24  42]
 [ 40   2  42  81  25   9  41   7  42  93]
 [ 55  25  50  99  42  25   4  36  17  42]
 [ 25   1  86   2  25  41  42 107  18  55]
 [ 93  36   2 101   2   7  55  25  16  59]
 [  7  42  36  25  21  26 100   6  41  16]]


In [11]:
train_on_gpu = torch.cuda.is_available()

In [26]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.4, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        self.lstm1 = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc1 = nn.Linear(n_hidden, int(n_hidden/2))
        self.fc2 = nn.Linear(int(n_hidden/2), int(n_hidden/2))
        self.fc3 = nn.Linear(int(n_hidden/2), len(self.chars))
      
    
    def forward(self, x, hidden):
        
        r_output, hidden = self.lstm1(x, hidden)
        
        out = self.dropout(r_output)
        
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [28]:
# define and print the net
n_hidden=1024
n_layers=5

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm1): LSTM(109, 1024, num_layers=5, batch_first=True, dropout=0.4)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=109, bias=True)
)


In [29]:
def train(net, data, epochs=10, batch_size=10, seq_length=10, lr=0.001, clip=5, val_frac=0.15, print_every=10):
    
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [30]:
batch_size = 128
seq_length = 10
n_epochs = 70

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 1/70... Step: 50... Loss: 3.4128... Val Loss: 3.4643
Epoch: 2/70... Step: 100... Loss: 3.0587... Val Loss: 3.1076
Epoch: 2/70... Step: 150... Loss: 2.7304... Val Loss: 2.7201
Epoch: 3/70... Step: 200... Loss: 2.5849... Val Loss: 2.5793
Epoch: 4/70... Step: 250... Loss: 2.4357... Val Loss: 2.4940
Epoch: 4/70... Step: 300... Loss: 2.3664... Val Loss: 2.3872
Epoch: 5/70... Step: 350... Loss: 2.2948... Val Loss: 2.3186
Epoch: 6/70... Step: 400... Loss: 2.2119... Val Loss: 2.2166
Epoch: 6/70... Step: 450... Loss: 2.0461... Val Loss: 2.1320
Epoch: 7/70... Step: 500... Loss: 2.0327... Val Loss: 2.0771
Epoch: 8/70... Step: 550... Loss: 1.9828... Val Loss: 2.0311
Epoch: 8/70... Step: 600... Loss: 1.8668... Val Loss: 1.9707
Epoch: 9/70... Step: 650... Loss: 1.8193... Val Loss: 1.9362
Epoch: 10/70... Step: 700... Loss: 1.6953... Val Loss: 1.9234
Epoch: 10/70... Step: 750... Loss: 1.7355... Val Loss: 1.8849
Epoch: 11/70... Step: 800... Loss: 1.6648... Val Loss: 1.8923
Epoch: 12/70... Step: 

In [31]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [33]:
def sample(net, size, prime='হিমু ', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [34]:
print(sample(net, 1000, prime='হিমু ', top_k=5))

হিমু শুরু করুন।’

	আমি বললাম, আপনার অফিসে গাষ করেছি। আজ হল আমার ঘুম-দিবস।’

	জীবনবাবু নড়লেন না। চুপচাপ দাঁড়িয়ে রইলেন। আমি বললাম, আরো কিছু বলবেন?

	‘জ্বি-না।’

	‘আপনার কী ধারণা ডাক্তার সাহেব? সে বলল, মানুষটার জন্যে হয়তো বয়স বেশি লাগছে? কেন জানি মাথায় ভোঁতা ধরণের যন্ত্রনা হচ্ছে। চিঠি পাঠিয়েছে ইংরেজিতে টেই মেয়ে। এখন সেটা শুরু হয়েছে। আমি মুহিব সাহেব মশারির ডাণ্ডা খুলি না।’

	‘আমার কাছে কলম আছে।’

	‘আপনার কী ধারণা ডাক্তার সাহেব?’

	‘আপনার সঙ্গে খানিকক্ষন কথাবার্তা না। একটু বসুন। আপনি কী চান নাইয়া দেরি করেন। আমি বললাম, আপনি কেমন আছেন?’

	এমা যারামন্য পানি ফেলেছে। তাকে হাসপাতালে নিয়ে মাথা ব্যান্ডেজ করে বাসায়ে পৌঁছে েন্টের আসছে হয় মাথাধরার যেলিফোন শুরু করবেন।’

	‘আপনার দাদীমা ভাল আছেন?’

	‘হ্যাঁ, ভালই আছেন।’

	‘কবে আসবেন?’

	‘খাওয়াদাওয়া করছে?’

	‘প্রথম দিন কিছু খাননি। রাতে একটা মুখ লাল হত। খামার সামনে আমার জন্যে মেসে সকালবেলা চা হয় না। চা খেতে রাস্তার ওপাশে ক্যান্টিনে যেতে ইচ্ছা করে কাউকে কিছু বলছেন? তাস ওদেরকে কি না-খেলতে বলব?’

	‘না।’

	‘আমি অবশ্যি মা’কে খুন হতে দেখিনি। তেমন কোনো প্রমাণও পাই